In [1]:
# Imports
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np
from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import bokeh.io
import bokeh.plotting

# Produce 2N pt 2 Simulations #

In [2]:
###### Create CRN of Produce 2N
Prod2_off = Species("Prod2")
#rR1 = Species("rR1")
Broccoli_Aptamer = Species("BrocApt")
#dA1 = Species("dA1" )
DFHBI = Species("DFHBI")

Prod2 = Genelet(Prod2_off, transcript= Broccoli_Aptamer , activator= "dA1"  , inhibitor= "rR1", 
                rnap="RNAP", rnaseH="RNAseH")
M_Prod2 = Mixture(name = "Produce2_test", components = [Prod2], parameter_file = "default_parameters.txt")
repr(M_Prod2)
CRN_Prod2 = M_Prod2.compile_crn()

#rxn1 = ([Broccoli_Aptamer, DFHBI_dye], [ComplexSpecies([Broccoli_Aptamer, DFHBI_dye], name = "Broc_flu"), k = 9.96e-2)

rxn1 = Reaction([Broccoli_Aptamer,DFHBI], [ComplexSpecies([Broccoli_Aptamer, DFHBI], name="Flu1")], k = 9.96e-5)
#rxn2 = Reaction([ComplexSpecies([Broccoli_Aptamer, DFHBI], name="Flu1")], [Broccoli_Aptamer, DFHBI], k = 9.96e-10)
CRN_Prod2.add_reactions([rxn1])

# Write SBML file
#CRN_Prod2.write_sbml_file('Prod2_CRN.xml')

print(CRN_Prod2.pretty_print())

Species (11) = {0. Prod2_OFF, 1. complex[Prod2_OFF:dna[dA1]], 2. BrocApt, 3. dna[dA1], 4. rna[rR1], 5. protein[RNAP], 6. protein[RNAseH], 7. complex[rna[rR1]:dna[dA1]], 8. complex[complex[Prod2_OFF:dna[dA1]]:protein[RNAP]], 9. complex[Prod2_OFF:protein[RNAP]], 10. complex[complex[rna[rR1]:dna[dA1]]:protein[RNAseH]]}
Reactions (10) = [
0. Prod2_OFF + dna[dA1] --> complex[Prod2_OFF:dna[dA1]]        
        massaction: k_f(Prod2_OFF,dna[dA1])=0.00394*Prod2_OFF*dna[dA1]
1. complex[Prod2_OFF:dna[dA1]] + rna[rR1] --> Prod2_OFF + complex[rna[rR1]:dna[dA1]]        
        massaction: k_f(complex[Prod2_OFF:dna[dA1]],rna[rR1])=0.0696*complex[Prod2_OFF:dna[dA1]]*rna[rR1]
2. dna[dA1] + rna[rR1] --> complex[rna[rR1]:dna[dA1]]        
        massaction: k_f(dna[dA1],rna[rR1])=0.0696*dna[dA1]*rna[rR1]
3. complex[Prod2_OFF:dna[dA1]] + protein[RNAP] <--> complex[complex[Prod2_OFF:dna[dA1]]:protein[RNAP]]        
        massaction: k_f(complex[Prod2_OFF:dna[dA1]],protein[RNAP])=38.857142857142854*co

In [3]:
# BioSCRAPE simulation of Produce2N with No repressors#
#Close to Schaffter and Schulman's paper#

io = {"Prod2_OFF": 0.035, "dna_dA1": 1, "rna_rR1": 0, "DFHBI": 15, 
      "protein_RNAseH":0, "protein_RNAP":15}
# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'
timepoints = np.linspace(0, 28800, 1000)
R = CRN_Prod2.simulate_with_bioscrape(timepoints, initial_condition_dict = io)
timepoints = timepoints/3600

bokeh.io.output_notebook()
p = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,
)
p.circle(timepoints, R["Prod2_OFF"], legend_label = "OFF Produce 2", color = "orange")
p.circle(timepoints, R["complex_Prod2_ON"], legend_label = "ON Produce 2", color = "red")
p.legend.click_policy="hide"

#v = bokeh.plotting.figure(
#    plot_width = 400, 
#    plot_height = 300,
#    x_axis_label = x,
#    y_axis_label = y,)
#v.circle(timepoints, R["complex_Prod2_ON"], legend_label = "ON Produce 2" , color = "red")
#v.legend.click_policy="hide"
#v.legend.location = "center_right"

r = bokeh.plotting.figure(
     plot_width = 300, 
     plot_height = 300,
     x_axis_label = x,
     y_axis_label = y,)
r.circle(timepoints, R["complex_Flu1"], legend_label = "Fluoresence", color = "darkgreen")
r.legend.location = "bottom_right"

s = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
s.circle(timepoints, R["BrocApt"], legend_label = "Broc Apt", color = "green")
s.legend.click_policy="hide"
s.legend.location = "top_left"

# t = bokeh.plotting.figure(
#     plot_width = 300, 
#     plot_height = 300,
#     x_axis_label = x,
#     y_axis_label = y,)
# t.circle(timepoints, R["DFHBI"], legend_label = "DFHBI molecules", color = "lightgreen")
# t.legend.click_policy="hide"
# t.legend.location = "top_right"

bokeh.io.show(row (p, s, r))
warnings.filterwarnings("ignore")

C:\Users\Jeremiah\anaconda3\lib\site-packages\html5lib\_trie\_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
C:\Users\Jeremiah\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Jeremiah\anaconda3\lib\site-packages\biocrnpyler\chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: DFHBI, complex_Flu1, 
  initial_condition_dict = initial_condition_dict)


Loading BokehJS ...

In [4]:
a = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
a.circle(timepoints, R["dna_dA1"], legend_label = "dA1", color = "blue")
a.legend.click_policy="hide"
a.legend.location = "top_right"

b = bokeh.plotting.figure(
    plot_width = 300, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
b.circle(timepoints, R["protein_RNAP"], legend_label = "RNAP", color = "green")

b.legend.click_policy="hide"
b.legend.location = "top_right"

bokeh.io.show(row (a, b))
warnings.filterwarnings("ignore")

In [6]:
c = bokeh.plotting.figure(
    plot_width = 500, 
    plot_height = 300,
    x_axis_label = x,
    y_axis_label = y,)
c.circle(timepoints, R["complex_Prod2_ON_protein_RNAP"], legend_label = "Produce 2 ON w/ RNAP", color = "purple")
c.circle(timepoints, R["complex_Prod2_ON"], legend_label = "ON Produce 2" , color = "red")
c.circle(timepoints, R["complex_Prod2_OFF_protein_RNAP"], legend_label = "ON Produce 2 OFF RNAP" , color = "blue")
c.legend.click_policy="hide"
c.legend.location = "center_right"

bokeh.io.show(row(c))